In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
import pandas as pd
import matplotlib.pyplot as plt
from zipfile import ZipFile
import os

In [10]:
uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path = keras.utils.get_file(origin=uri, fname="jena_climate_2009_2016.csv.zip")
zip_file = ZipFile(zip_path)
zip_file.extractall()
csv_path = "jena_climate_2009_2016.csv"

df = pd.read_csv(csv_path)

13582336/13568290 [==============================] - 1s 0us/step


In [14]:
titles = [
    "Pressure",
    "Temperature",
    "Temperature in Kelvin",
    "Temperature (dew point)",
    "Relative Humidity",
    "Saturation vapor pressure",
    "Vapor pressure",
    "Vapor pressure deficit",
    "Specific humidity",
    "Water vapor concentration",
    "Airtight",
    "Wind speed",
    "Maximum wind speed",
    "Wind direction in degrees",
]

feature_keys = [
    "p (mbar)",
    "T (degC)",
    "Tpot (K)",
    "Tdew (degC)",
    "rh (%)",
    "VPmax (mbar)",
    "VPact (mbar)",
    "VPdef (mbar)",
    "sh (g/kg)",
    "H2OC (mmol/mol)",
    "rho (g/m**3)",
    "wv (m/s)",
    "max. wv (m/s)",
    "wd (deg)",
]

colors = [
    "blue",
    "orange",
    "green",
    "red",
    "purple",
    "brown",
    "pink",
    "gray",
    "olive",
    "cyan",
]

date_time_key = "Date Time"

In [12]:
split_fraction = 0.715
train_split = int(split_fraction * int(df.shape[0]))
step = 6

past = 720
future = 72
learning_rate = 0.001
batch_size = 256
epochs = 10


def normalize(data, train_split):
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data - data_mean) / data_std

In [15]:
print(
    "The selected parameters are:",
    ", ".join([titles[i] for i in [0, 1, 5, 7, 8, 10, 11]]),
)
selected_features = [feature_keys[i] for i in [0, 1, 5, 7, 8, 10, 11]]
features = df[selected_features]
features.index = df[date_time_key]
features.head()

features = normalize(features.values, train_split)
features = pd.DataFrame(features)
features.head()

train_data = features.loc[0 : train_split - 1]
val_data = features.loc[train_split:]

The selected parameters are: Pressure, Temperature, Saturation vapor pressure, Vapor pressure deficit, Specific humidity, Airtight, Wind speed


In [16]:
start = past + future
end = start + train_split

x_train = train_data[[i for i in range(7)]].values
y_train = features.iloc[start:end][[1]]

sequence_length = int(past / step)

In [17]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

In [18]:
x_end = len(val_data) - past - future

label_start = train_split + past + future

x_val = val_data.iloc[:x_end][[i for i in range(7)]].values
y_val = features.iloc[label_start:][[1]]

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)


for batch in dataset_train.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (256, 120, 7)
Target shape: (256, 1)


Training with ANN.

In [19]:
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = Linear(32)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 120, 7)]          0         
                                                                 
 linear_1 (Linear)           (None, 120, 32)           256       
                                                                 
 dense (Dense)               (None, 120, 1)            33        
                                                                 
Total params: 289
Trainable params: 289
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val
)

Epoch 1/10
1172/1172 [==============================] - 14s 11ms/step - loss: 0.4197 - val_loss: 103.1771
Epoch 2/10
1172/1172 [==============================] - 13s 11ms/step - loss: 0.3982 - val_loss: 100.0180
Epoch 3/10
1172/1172 [==============================] - 13s 11ms/step - loss: 0.3939 - val_loss: 94.9508
Epoch 4/10
1172/1172 [==============================] - 13s 11ms/step - loss: 0.3917 - val_loss: 90.0621
Epoch 5/10
1172/1172 [==============================] - 14s 12ms/step - loss: 0.3905 - val_loss: 83.9553
Epoch 6/10
1172/1172 [==============================] - 13s 11ms/step - loss: 0.3896 - val_loss: 82.5686
Epoch 7/10
1172/1172 [==============================] - 13s 11ms/step - loss: 0.3889 - val_loss: 83.9479
Epoch 8/10
1172/1172 [==============================] - 14s 12ms/step - loss: 0.3881 - val_loss: 81.4972
Epoch 9/10
1172/1172 [==============================] - 13s 11ms/step - loss: 0.3877 - val_loss: 80.9430
Epoch 10/10
1172/1172 [==============================

Training with LSTM.

In [33]:
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out, _ = LSTM(32)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 120, 7)]          0         
                                                                 
 lstm_5 (LSTM)               ((None, 120, 32),         5120      
                              (None, 240, 32))                   
                                                                 
 dense_3 (Dense)             (None, 120, 1)            33        
                                                                 
Total params: 5,153
Trainable params: 5,153
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val
)

Epoch 1/10
1172/1172 [==============================] - 37s 31ms/step - loss: 0.4462 - val_loss: 0.3664
Epoch 2/10
1172/1172 [==============================] - 38s 32ms/step - loss: 0.3845 - val_loss: 0.3641
Epoch 3/10
1172/1172 [==============================] - 36s 31ms/step - loss: 0.3834 - val_loss: 0.3624
Epoch 4/10
1172/1172 [==============================] - 36s 31ms/step - loss: 0.3825 - val_loss: 0.3614
Epoch 5/10
1172/1172 [==============================] - 36s 31ms/step - loss: 0.3817 - val_loss: 0.3606
Epoch 6/10
1172/1172 [==============================] - 37s 32ms/step - loss: 0.3809 - val_loss: 0.3598
Epoch 7/10
1172/1172 [==============================] - 39s 33ms/step - loss: 0.3802 - val_loss: 0.3592
Epoch 8/10
1172/1172 [==============================] - 38s 33ms/step - loss: 0.3794 - val_loss: 0.3586
Epoch 9/10
1172/1172 [==============================] - 39s 33ms/step - loss: 0.3787 - val_loss: 0.3581
Epoch 10/10
1172/1172 [==============================] - 36s 31m

# Code Repo

### Code Snippet A: Adam Optimizer.

In [2]:
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
var1 = tf.Variable(10.0)
loss = lambda: (var1 ** 2)/2.0       # d(loss)/d(var1) == var1
step_count = opt.minimize(loss, [var1]).numpy()
# The first step is `-learning_rate*sign(grad)`
var1.numpy()

9.9

### Code Snippet B: Layer Interface.

In [4]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

### Code Snippet C: Naive LSTM

In [30]:
'''
A Naive LSTM Cell Implementation.

@link tensorflow/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py#BasicLSTMCell
@link https://github.com/piEsposito/pytorch-lstm-by-hand/blob/master/LSTM.ipynb
@date MAR-29-2022
@note straightforward, in sacrifice of efficiency
'''
class LSTM(keras.layers.Layer):
    def __init__(self, units=32):
        super(LSTM, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.W_i = self.add_weight(shape=(input_shape[-1], self.units), initializer="random_normal", trainable=True)
        self.U_i = self.add_weight(shape=(self.units, self.units), initializer="random_normal", trainable=True)
        self.b_i = self.add_weight(shape=(self.units, ), initializer="random_normal", trainable=True)

        self.W_f = self.add_weight(shape=(input_shape[-1], self.units), initializer="random_normal", trainable=True)
        self.U_f = self.add_weight(shape=(self.units, self.units), initializer="random_normal", trainable=True)
        self.b_f = self.add_weight(shape=(self.units, ), initializer="random_normal", trainable=True)

        self.W_c = self.add_weight(shape=(input_shape[-1], self.units), initializer="random_normal", trainable=True)
        self.U_c = self.add_weight(shape=(self.units, self.units), initializer="random_normal", trainable=True)
        self.b_c = self.add_weight(shape=(self.units, ), initializer="random_normal", trainable=True)

        self.W_o = self.add_weight(shape=(input_shape[-1], self.units), initializer="random_normal", trainable=True)
        self.U_o = self.add_weight(shape=(self.units, self.units), initializer="random_normal", trainable=True)
        self.b_o = self.add_weight(shape=(self.units, ), initializer="random_normal", trainable=True)

    def call(self, inputs, init_states=None):
        bs = inputs.shape[1]
        one = constant_op.constant(1, dtype=dtypes.int32)
        sigmoid = math_ops.sigmoid
        tanh = math_ops.tanh
        if init_states is None:
            h_t, c_t = (
                tf.zeros((bs, self.units)),
                tf.zeros((bs, self.units)),
            )
        else:
            h_t, c_t = array_ops.split(value=init_states, num_or_size_splits=2, axis=one)

        x_t = inputs

        i_t = sigmoid(x_t @ self.W_i + h_t @ self.U_i + self.b_i)
        f_t = sigmoid(x_t @ self.W_f + h_t @ self.U_f + self.b_f)
        g_t = tanh(x_t @ self.W_c + h_t @ self.U_c + self.b_c)
        o_t = sigmoid(x_t @ self.W_o + h_t @ self.U_o + self.b_o)
        c_t = f_t * c_t + i_t * g_t
        h_t = o_t * tanh(c_t)
            
        new_state = array_ops.concat([h_t, c_t], 1)
        return h_t, new_state